## Configure the CPDCTL for coneecting to a cluster

In [16]:
SPACE_NAME="p-test-space"
DEPLOY_NAME="p-test-deploy"

In [2]:
cpdctl config users list
cpdctl config users set qa-user --username=admin --password=password

Name      Username   
qa-user   admin   


In [10]:
cpdctl config profiles set qa-cluster --user qa-user --url https://zen-cpd-zen.apps.pwh.ocp.csplab.local

In [7]:
cpdctl config contexts set qa-context qa-cluster


In [8]:
cpdctl spaces list | grep prod-space | awk 'END{print NR}'

1


In [3]:
cpdctl spaces list 

...
ID                                     Name         Created                    Tags   
89642363-4593-4db5-bf02-f2141155963c   prod-space   2020-10-20T17:28:20.894Z   []   


In [21]:
str1=$(cpdctl spaces list | grep $SPACE_NAME | awk 'END{print NR}')
if [[ $str1 != 1 ]]; then
 echo "deleting extra spaces";
 cpdctl spaces list | grep $SPACE_NAME | awk 'NR>1{print $1}' | xargs -I{} cpdctl spaces delete --space_id {}
fi
if [[ $str1 == 0 ]]; then
 echo "creating space"
cpdctl spaces create --name $SPACE_NAME
 else
   cpdctl spaces list
   export SPACE_ID=$(cpdctl spaces list | grep $SPACE_NAME | awk '{ print $1}')
   echo $SPACE_ID
fi

...
ID                                     Name           Created                    Tags   
713f8e3e-4a37-4961-8d08-c8cacfaf469d   p-test-space   2020-10-20T17:42:46.956Z   []   
fc235249-e514-4c39-9f53-3fcae22b6996   prod-space     2020-10-20T17:33:14.777Z   []   
713f8e3e-4a37-4961-8d08-c8cacfaf469d


In [22]:
models=$(cpdctl wml models list --space-id $SPACE_ID | awk 'NR>1')
if [[ $models == "Nothing to show." ]]; then
   echo "No models found, creating model"
   cpdctl wml models create --name mlopsmodel --space-id $SPACE_ID --software-spec '{"name": "scikit-learn_0.22-py3.6"}' --type 'scikit-learn_0.22'
   
   export MODEL_ID=$(cpdctl wml models list --space-id $SPACE_ID | grep mlopsmodel | awk '{ print $1}')
   cpdctl wml models upload-content --body mlopsmodelpipeline.tar.gz --model-id $MODEL_ID --content-format 'binary' --space-id $SPACE_ID --output json

 else
   cpdctl wml models list --space-id $SPACE_ID
   export MODEL_ID=$(cpdctl wml models list --space-id $SPACE_ID | grep mlopsmodel | awk '{ print $1}')
   echo $MODEL_ID
   cpdctl wml models upload-content --body mlopsmodelpipeline.tar.gz --model-id $MODEL_ID --content-format 'binary' --space-id $SPACE_ID --output json
   
fi

No models found, creating model
...
              
ID:        9401811f-8e3d-45d5-8540-b8266b7189e3   
Name:      mlopsmodel   
Created:   2020-10-20T18:00:19.002Z   
Type:      scikit-learn_0.22   
Tags:      []   
null


In [23]:
deployments=$(cpdctl wml deployments list --space-id $SPACE_ID |awk 'NR>1')
echo $deployments
if [[ $deployments == "Nothing to show." ]]; then
echo "Creating Deployment"
cpdctl wml deployments create --name $DEPLOY_NAME --online '{"description": "mlopsdeploy"}' --asset '{"id": "'"$MODEL_ID"'"}' --space-id $SPACE_ID
fi
DEPLOY_ID=$(cpdctl wml deployments list --space-id $SPACE_ID | grep $DEPLOY_NAME| awk 'NR>2{print $1}')
echo $DEPLOY_ID

Nothing to show.
Creating Deployment
...
                  
ID:            6f72017e-af84-4186-bc15-bf44ae5a3ff7   
Name:          p-test-deploy   
Created:       2020-10-20T18:00:40.501Z   
Description:      
State:         ready   
Tags:          []   



In [24]:
cpdctl wml deployments list --space-id $SPACE_ID

...
ID                                     Name            Created                    Description   State   Tags   
6f72017e-af84-4186-bc15-bf44ae5a3ff7   p-test-deploy   2020-10-20T18:00:40.501Z                 ready   []   


In [30]:
cpdctl wml deployments get --deployment-id '6f72017e-af84-4186-bc15-bf44ae5a3ff7' --space-id $SPACE_ID --output json

{
  "entity": {
    "asset": {
      "id": "9401811f-8e3d-45d5-8540-b8266b7189e3"
    },
    "custom": {},
    "description": "",
    "name": "p-test-deploy",
    "online": {},
    "space_id": "713f8e3e-4a37-4961-8d08-c8cacfaf469d",
    "status": {
      "online_url": {
        "url": "https://zen-cpd-zen.apps.pwh.ocp.csplab.local/v4/deployments/6f72017e-af84-4186-bc15-bf44ae5a3ff7/predictions"
      },
      "state": "ready"
    }
  },
  "metadata": {
    "created_at": "2020-10-20T18:00:40.501Z",
    "description": "",
    "id": "6f72017e-af84-4186-bc15-bf44ae5a3ff7",
    "modified_at": "2020-10-20T18:00:40.501Z",
    "name": "p-test-deploy",
    "space_id": "713f8e3e-4a37-4961-8d08-c8cacfaf469d"
  }
}


In [35]:
asset=$(echo "'"$(cat asset.json)"'")

In [18]:
echo $DEPLOY_ID

In [10]:
echo $DEPLOY_ID

dde25682-e6cc-4dfe-9ce9-0c9f4a7dc55a


In [20]:
cpdctl wml deployments compute-predictions --output json --deployment-id $DEPLOY_ID --input-data '[{"values":[[0.0,0.0,2.0,10.0,7.0,0.0,0.0,0.0,0.0,0.0,14.0,16.0,16.0,15.0,1.0,0.0,0.0,4.0,16.0,7.0,3.0,16.0,7.0,0.0,0.0,5.0,16.0,10.0,7.0,16.0,4.0,0.0,0.0,0.0,5.0,14.0,14.0,16.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,2.0,0.0,0.0,0.0,4.0,7.0,7.0,16.0,2.0,0.0,0.0,0.0,5.0,12.0,16.0,12.0,0.0,0.0]]}]'

{
  "predictions": [
    {
      "fields": [
        "prediction",
        "probability"
      ],
      "values": [
        [
          9,
          [
            0.0019493161921729843,
            0.001304343320243037,
            0.0007079363358815012,
            0.0008947145181351228,
            0.0010433292618768476,
            0.003976107818992792,
            0.0011521134168434308,
            0.0008233651358512275,
            0.005568478988915326,
            0.9825802950110881
          ]
        ]
      ]
    }
  ]
}


In [15]:
FIELDS=$(head -n1  data.csv | awk -F"," '{ $65=$66=$67="";print $0}'  | rev | cut -c 2- | rev |  awk -v OFS="," '{for (i=1; i<=NF; ++i){$i="\""$i"\""}}1')

In [16]:
echo $FIELDS

"Column0","Column1","Column2","Column3","Column4","Column5","Column6","Column7","Column8","Column9","Column10","Column11","Column12","Column13","Column14","Column15","Column16","Column17","Column18","Column19","Column20","Column21","Column22","Column23","Column24","Column25","Column26","Column27","Column28","Column29","Column30","Column31","Column32","Column33","Column34","Column35","Column36","Column37","Column38","Column39","Column40","Column41","Column42","Column43","Column44","Column45","Column46","Column47","Column48","Column49","Column50","Column51","Column52","Column53","Column54","Column55","Column56","Column57","Column58","Column59","Column60","Column61","Column62","Column63"


In [22]:
for row in $(awk -F"," -v OFS="," 'NR>1{$66=$65=""; print}' datawoprob.csv | rev| cut -c 3- | rev); 
do payload='[{"fields": '"[$FIELDS] "', "values": '"[[$row]]"'}]';
echo $payload;
cpdctl wml  deployments compute-predictions --deployment-id $DEPLOY_ID --input-data "$payload" --output json
 done

[{"fields": ["Column0","Column1","Column2","Column3","Column4","Column5","Column6","Column7","Column8","Column9","Column10","Column11","Column12","Column13","Column14","Column15","Column16","Column17","Column18","Column19","Column20","Column21","Column22","Column23","Column24","Column25","Column26","Column27","Column28","Column29","Column30","Column31","Column32","Column33","Column34","Column35","Column36","Column37","Column38","Column39","Column40","Column41","Column42","Column43","Column44","Column45","Column46","Column47","Column48","Column49","Column50","Column51","Column52","Column53","Column54","Column55","Column56","Column57","Column58","Column59","Column60","Column61","Column62","Column63"] , "values": [[0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,13.0,15.0,10.0,15.0,5.0,0.0,0.0,3.0,15.0,2.0,0.0,11.0,8.0,0.0,0.0,4.0,12.0,0.0,0.0,8.0,8.0,0.0,0.0,5.0,8.0,0.0,0.0,9.0,8.0,0.0,0.0,4.0,11.0,0.0,1.0,12.0,7.0,0.0,0.0,2.0,14.0,5.0,10.0,12.0,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0]]}]
FAI

In [24]:
cpdctl wml deployments list --space-id $SPACE_ID

...
FAILED


trace           message                                                                                                   code   
14lxc5j7iiiq7   Failed to fetch space details : '/v4/spaces/1552222d-2762-4006-847d-d1d675ff6f66' for the current user.   space_validation_failure   


: 1